In [50]:
%%time
import re
import pathlib
from bs4 import BeautifulSoup as BS  
import glob
import os

# fichiers viewer : https://www.gleif.org/assets/components/xbrl-viewer/gleif-annual-report-2019/ixbrl-viewer.signed4.htm
# August 2020
# Parse iXBRL reports, published as .html or .xhtml file from local
# Minimal example: Use of imports is kept to an absolute minimum 

# Define path and import html or xhtml from local
try:
   os.mkdir('mon_csv')
except:
    pass

filepath = str(pathlib.Path.cwd())
filename = glob.glob('Fichiers_Test/*.xhtml')
Mypath = filepath+"\\"+filename[1]
mycsvpath = str(os.getcwd() + '\Old_version\\mon_csv')
print(Mypath)
print(mycsvpath)
# To read from online source just change this part
soup = BS(open(Mypath, "rb"), "html.parser")
#soup = BS(open(mypath+"evonik.xhtml", "rb"), "html.parser")

# Set up a file to store the results
outFile = open(mycsvpath+"\\"+"myxbrltable_test.csv", 'w', newline='\r', encoding='utf8')
outFile.write("position;tag;type;Valeur_numérique;Unite;Date;identi;dim;form \n")


# Loop over elements in html/xhtml
position = 0

for c, element in enumerate(soup.find_all()):
    # Iterate over all XBLR content (identif. by "contextref")
    if "contextref" in element.attrs:
        print(element.attrs[''])
        # See all relevant attributes of an XBRL tag
        #print(element.attrs)

        #################################################
        # Retrieve content per tag

        context_ref= element.attrs['contextref']
        # Name of entry (aka name of XBLR tag)
        tag = str(element.attrs['name'])
        
        # Tagtype (standard ifrs vs. custom)
        print(tag)
        if "ifrs-full" in tag:
            tagtype="IFRS"
        else:
            tagtype="Custom"        
        tag = tag[tag.rfind(":")+1:]
        tag = re.findall('[a-zA-Z][^A-Z]*',tag)
        for i in range (len(tag)-1):
            tag[i+1] = tag[i+1].lower()
            
        tag = ' '.join(tag)

        # Get content (text in tag)
        # This is e.g. the numeric value of a position
        content = element.text
        content = content.replace('\n', '').replace('\r', '').lstrip()
        # Get unit (like USD or EUR)
        try:
            # Somewhere hidden at top of file
            #  <xbrli:unit id="ibee02267a6cf4a178ffe2b57731cf158_0f1b2a3d-0585-35e4-9bf4-b4f1337c8ffb">
            #  <xbrli:measure>iso4217:USD</xbrli:measure>
            unit = str(soup.find(id=element['unitref']).text)
            unit = unit.strip().replace('\n', '').replace('\r', '')
        except:
            unit = "nn"
        
        # Get time and identity reference 
        try:
            identi="nn"
            date1="nn"
            date2="nn"
            dim="nn"
            # print(soup.find(id=element['contextref']))
            for c in soup.find_all(id=element['contextref']):
                # To plain text
                res = c.text
                res = res.replace('\n', '').replace('\r', '').lstrip()
                # Find date by regex
                match = re.search(r'\d{4}-\d{2}-\d{2}', res)
                date1 = match.group()
                # Remove date from string
                res = res.replace(date1,"")
                # Find second date
                match = re.search(r'\d{4}-\d{2}-\d{2}', res)
                date2 = match.group()

                # Remove second date
                res = res.replace(date2,"")
                # Find ifrs-ref if present
                dim = res[res.find("ifrs"):]
                if len(dim)>2:
                    res = res.replace(dim,"")
                else:
                    dim = "nn"
                

                    
                # Remove ifrs reference if prsent
                identi = res
        except:
            pass

        # Internal ID of tag
        try:
            cid = str(element.attrs['id'])
        except:
            cid = "nn"
        
        
        # Number of decimals
        try:
            decimals = str(element.attrs['decimals'])
        except:
            decimals = "nn"

        # Sign of numeric value (only if negative, aka. "minus")
        try:
            sign = str(element.attrs['sign'])
        except:
            sign = ""

        # Format of numerical content (decimal separator et al.)
        # https://www.xbrl.org/Specification/inlineXBRL-transformationRegistry/REC-2015-02-26/inlineXBRL-transformationRegistry-REC-2015-02-26.html
        try:
            form = str(element.attrs['format'])
        except:
            form = "nn"
        
        # Scale of numerical content
        # E.g. when figures are expressed in "million EUR"
        # Numerical values to be transformed by: numcontent * 10**int(scale)
        try:
            scale = str(element.attrs['scale'])
        except:
            scale = "nn"
        
        #################################################
        ## Prepare content
        content = sign + content
        # Check if content is a number if true assign numerical value to separate variable
        numcontent = "0"
        try:
            x=content.replace(",","").replace(".","").replace("-","")
            if x.isdigit()==True: 
                # If number... do formating
                

                # CASE: Comma decimal separatot to UK/US
                if form=="ixt:numcomma" or form=="ixt:numdotcomma" or form=="numspacecomma":
                    numcontent = content.replace(".","")
                    numcontent = content.replace(",",".")
                # CASE: UK/US format
                if form=="ixt:numcommadot" or form=="ixt:numspacedot" or form=="ixt:num-dot-decimal":
                    numcontent = content.replace(",","")
                else:
                    pass
                
                # As float
                numcontent = float(numcontent)
                # Scale (using "10 to the power of" scale)
                if scale!="nn":
                    numcontent = numcontent * 10**int(scale)
                    numcontent = str(numcontent)
            position = position+1
        except:
            pass
        
        date3 = date2
        if date3 == "nn":
            date3 = date1;

        
        # Write to csv file
       # outFile.write(str(position) +";"+ tag +";"+ tagtype +";"+ str(numcontent) +";"+ unit +";"+date3 + ";"  +identi +";"+dim +";"+form +"\n")

        # print(type(outFile))
outFile.close()



C:\Users\maxen\OneDrive\Bureau\ISEP\projet de fin de parcours\FinHarmony\Fichiers_Test\gleif-19ar.signed4.xhtml
C:\Users\maxen\OneDrive\Bureau\ISEP\projet de fin de parcours\FinHarmony\Old_version\mon_csv
ifrs-full:NameOfReportingEntityOrOtherMeansOfIdentification
ifrs-full:DomicileOfEntity
ifrs-full:DescriptionOfNatureOfEntitysOperationsAndPrincipalActivities
ifrs-full:LicenceFeeIncome
ifrs-full:LicenceFeeIncome
ifrs-full:WagesAndSalaries
ifrs-full:WagesAndSalaries
gleif:SocialContributionsAndExpensesForPensionsAndCare
gleif:SocialContributionsAndExpensesForPensionsAndCare
ifrs-full:EmployeeBenefitsExpense
ifrs-full:EmployeeBenefitsExpense
ifrs-full:OtherExpenseByNature
ifrs-full:OtherExpenseByNature
ifrs-full:OtherIncome
ifrs-full:OtherIncome
ifrs-full:DepreciationAndAmortisationExpense
ifrs-full:DepreciationAndAmortisationExpense
ifrs-full:ProfitLossFromOperatingActivities
ifrs-full:ProfitLossFromOperatingActivities
ifrs-full:RevenueFromGovernmentGrants
ifrs-full:RevenueFromGovernme

In [43]:
glob.glob('Fichiers_Test/*.xhtml')

['Fichiers_Test\\fichier.xhtml',
 'Fichiers_Test\\gleif-19ar.signed4.xhtml',
 'Fichiers_Test\\gleif-annual-report-2018-viewer.xhtml',
 'Fichiers_Test\\gleif-annual-report-2018.signed.signed.signed.signed.xhtml']

In [44]:
import pandas as pd 

df = pd.read_csv(mycsvpath+'\\myxbrltable2019.csv',sep=';')
df

,position,tag,type,Valeur_numérique,Unite,Date,identi,dim,form
0,1,Name of reporting entity or other means of ide...,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
1,2,Domicile of entity,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
2,3,Description of nature of entitys operations an...,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
3,4,Licence fee income,IFRS,14516715.0,iso4217:USD,2019-12-31,506700GE1G29325QX363,nn,ixt:num-dot-decimal
4,5,Licence fee income,IFRS,17555899.0,iso4217:USD,2018-12-31,506700GE1G29325QX363,nn,ixt:num-dot-decimal
...,...,...,...,...,...,...,...,...,...
158,159,Equity,IFRS,14186580.0,iso4217:USD,2019-12-31,nn,nn,ixt:num-dot-decimal
159,160,Country of incorporation,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
160,161,Principal place of business,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
161,162,Legal form of entity,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn


In [37]:
df['tag'].value_counts()

Equity                                                                                                                                           14
Profit loss                                                                                                                                      12
Licence fee income                                                                                                                               12
Comprehensive income                                                                                                                             10
Other comprehensive income                                                                                                                       10
Cash and cash equivalents                                                                                                                         6
Other expense by nature                                                                                         

In [14]:
import pandas as pd
df = pd.read_csv('Fichiers_Test\myxbrltable.csv', sep=';')
df

,position,tag,type,Valeur_numérique,Unite,Date,identi,dim,form
0,1,Name of reporting entity or other means of ide...,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
1,2,Domicile of entity,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
2,3,Description of nature of entitys operations an...,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
3,4,Licence fee income,IFRS,14516715.0,iso4217:USD,2019-12-31,506700GE1G29325QX363,nn,ixt:num-dot-decimal
4,5,Licence fee income,IFRS,17555899.0,iso4217:USD,2018-12-31,506700GE1G29325QX363,nn,ixt:num-dot-decimal
...,...,...,...,...,...,...,...,...,...
321,322,Equity,IFRS,14186580.0,iso4217:USD,2019-12-31,nn,nn,ixt:num-dot-decimal
322,323,Country of incorporation,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
323,324,Principal place of business,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn
324,325,Legal form of entity,IFRS,0.0,nn,2019-12-31,506700GE1G29325QX363,nn,nn


In [11]:
os.listdir()

['.DS_Store',
 '.git',
 '.ipynb_checkpoints',
 'app_dash.py',
 'assets',
 'convert.py',
 'Converter_xbrl.ipynb',
 'Fichiers_Test',
 'FinHarmony maquette.epgz',
 'mon_csv',
 'Old version',
 'Présentation Finale - Knowledge Management.pdf',
 'README.md',
 'requirements.txt',
 '__pycache__']